### Load Data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
!wget 'https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip' 

--2020-07-05 09:42:53--  https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3380245855 (3.1G) [application/zip]
Saving to: ‘MURA-v1.1.zip’

MURA-v1.1.zip       100%[===================>]   3.15G  13.5MB/s    in 3m 57s  

2020-07-05 09:46:50 (13.6 MB/s) - ‘MURA-v1.1.zip’ saved [3380245855/3380245855]



In [ ]:
!unzip MURA-v1.1.zip 

Archive:  MURA-v1.1.zip
replace MURA-v1.1/train_labeled_studies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
drive_path='/content/drive/My Drive/MURA Challenge/data/'
path='/content/drive/My Drive/MURA Challenge/data/MURA-v1.1/'
colab_path='/content/MURA-v1.1/'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from tqdm import tqdm
from glob import glob
import tensorflow as tf
from tensorflow import keras

In [ ]:
# os.listdir(path)

In [ ]:
train_imgs_path=pd.read_csv(colab_path+'train_image_paths.csv',dtype=str,header=None,names=['Img_Path'])
train_labels=pd.read_csv(colab_path+'train_labeled_studies.csv',dtype=str,names=['Img_Path','Label'],header=None)
test_imgs_path=pd.read_csv(colab_path+'valid_image_paths.csv',dtype=str,header=None,names=['Img_Path'])
test_labels=pd.read_csv(colab_path+'valid_labeled_studies.csv',dtype=str,names=['Img_Path','Label'],header=None)

In [ ]:
def keep_dir(dir):
  x=list(dir.split('/'))
  x='/'.join(x[:5])
  x=x+'/'
  return x
def keep_dir2(dir):
  x=list(dir.split('/'))
  x='/'.join(x[3:5])
  x=x+'/'
  return x

In [ ]:
keep_dir('MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png')

'MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/'

In [ ]:
train_imgs_path['path_to_merge']=train_imgs_path['Img_Path'].apply(lambda x: keep_dir(x))
test_imgs_path['path_to_merge']=test_imgs_path['Img_Path'].apply(lambda x: keep_dir(x))

In [ ]:
train_data=train_imgs_path.merge(train_labels,how='inner',left_on='path_to_merge', right_on='Img_Path')
test_data=test_imgs_path.merge(test_labels,how='inner',left_on='path_to_merge', right_on='Img_Path')

In [ ]:
train_data['Body_Part']=train_data['Img_Path_x'].apply(lambda x: str(x.split('/')[2]))
test_data['Body_Part']=test_data['Img_Path_x'].apply(lambda x: str(x.split('/')[2]))
train_data['Patient/Study']=train_data['Img_Path_x'].apply(lambda x: str(x.split('/')[3:5]))
test_data['Patient/Study']=test_data['Img_Path_x'].apply(lambda x: str(x.split('/')[3:5]))
train_data['Patient/Study']=train_data['Img_Path_x'].apply(lambda x: keep_dir2(x) )
test_data['Patient/Study']=test_data['Img_Path_x'].apply(lambda x: keep_dir2(x) )
train_data.drop(columns=['path_to_merge','Img_Path_y'],inplace=True)
train_data.rename(columns={'Img_Path_x':'Img_Path'},inplace=True)
test_data.drop(columns=['path_to_merge','Img_Path_y'],inplace=True)
test_data.rename(columns={'Img_Path_x':'Img_Path'},inplace=True)

test_labels['Body_Part']=test_labels['Img_Path'].apply(lambda x: str(x.split('/')[2]))
test_labels['Patient/Study']=test_labels['Img_Path'].apply(lambda x:  keep_dir2(x))


# train_data.Img_Path=train_data.Img_Path.apply(lambda x: drive_path+x)
# test_data.Img_Path=test_data.Img_Path.apply(lambda x: drive_path+x)

In [ ]:
train_data.head()

,Img_Path,Label,Body_Part,Patient/Study
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1,XR_SHOULDER,patient00001/study1_positive/
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1,XR_SHOULDER,patient00001/study1_positive/
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1,XR_SHOULDER,patient00001/study1_positive/
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,1,XR_SHOULDER,patient00002/study1_positive/
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,1,XR_SHOULDER,patient00002/study1_positive/


### Define Evaluation metrics & Helper Functions


In [ ]:
import keras.backend as K
def recall(y_true, y_pred):
    
    """
    Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    
    """
    Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    Source
    ------
    https://github.com/fchollet/keras/issues/5400#issuecomment-314747992
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1(y_true, y_pred):
    
    """Calculate the F1 score."""
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r))


import gc

def clean_up(model):
    K.clear_session()
    del model
    gc.collect()


Using TensorFlow backend.


In [ ]:
# for visualizing the training progress of our models
%matplotlib inline
import matplotlib.pyplot as plt
def viz_history(hs_dict,metric):
  plt.style.use('dark_background')
  plt.rcParams['figure.figsize'] = [15, 8]
  plt.rcParams['font.size'] = 16
  plt.clf()
  for model in hs_dict.keys():
    plt.plot(hs_dict[model].history[metric],label='{0:s} train {1:s}'.format(model, metric))
    plt.plot(hs_dict[model].history['val_{0:s}'.format(metric)],label='{0:s} validation {1:s}'.format(model, metric))
    plt.ylabel(metric) 
    plt.xlabel('epoch')
    plt.legend()
  plt.show()

### Prep Data For Training




In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array 
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [ ]:
shuffled=train_data.sample(frac=1)

In [ ]:
msk = np.random.rand(len(shuffled)) <0.8

train = shuffled[msk]

val= shuffled[~msk]

In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import ResNet101
from keras.applications.densenet import DenseNet169
from keras.callbacks import *

In [ ]:
def train_model(
        optimizer,
        body_type='XR_ELBOW',
        input_shape=(224,224,3),
        output_activation='softmax',
        full_network_epochs=10,
        callbacks_list=None,
        verbose=1,
        class_weights=None,
        transfer_model=DenseNet169,
        pooling='avg'):
    

    train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      horizontal_flip=True,
  )

    val_datagen=ImageDataGenerator(
        rescale=1./255)


    train_generator=train_datagen.flow_from_dataframe(
        dataframe=train[train.Body_Part==body_type],
        directory=None,
        x_col="Img_Path",
        y_col="Label",
        weight_col=None,
        target_size=(224, 224),
        color_mode="rgb",
        classes=None,
        class_mode="categorical",
        batch_size=32,
        shuffle=False,
        seed=1,
        save_to_dir=None,
        save_prefix="",
        save_format="png",
        interpolation="nearest",
        validate_filenames=False,
    )

    val_generator=val_datagen.flow_from_dataframe(
        dataframe=val[val.Body_Part==body_type],
        directory=None,
        x_col="Img_Path",
        y_col="Label",
        weight_col=None,
        target_size=(224, 224),
        color_mode="rgb",
        classes=None,
        class_mode="categorical",
        batch_size=32,
        shuffle=False,
        seed=1,
        save_to_dir=None,
        save_prefix="",
        save_format="png",
        interpolation="nearest",
        validate_filenames=False
    )

    #model parameters for training
    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=val_generator.n//val_generator.batch_size



    base_model = transfer_model(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling=pooling,
    )

    x = base_model.output
    output = Dense(2, activation='softmax', name='predictions')(x)
    model = Model(inputs=base_model.input, output=output)

    for layer in base_model.layers:
            layer.trainable = True

    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',f1])
    if class_weights:
      hs_full_network = model.fit(
        train_generator,
        epochs=full_network_epochs,
        steps_per_epoch=STEP_SIZE_TRAIN,
        validation_data=val_generator,
        validation_steps=STEP_SIZE_VALID,
        callbacks=callbacks_list,
        class_weight=class_weights[body_type],
        verbose=1
      )
    else:
      hs_full_network = model.fit(
        train_generator,
        epochs=full_network_epochs,
        steps_per_epoch=STEP_SIZE_TRAIN,
        validation_data=val_generator,
        validation_steps=STEP_SIZE_VALID,
        callbacks=callbacks_list,
        verbose=1
      )

    print('Finished training full network.')
    print('------------------')

    return model, hs_full_network

In [ ]:
from sklearn.metrics import *
def evaluate_model(
    body_part='XR_ELBOW',
    model=None
):
  test_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      horizontal_flip=True,)
  
  test_generator=test_datagen.flow_from_dataframe(
      dataframe=test_data[test_data.Body_Part==body_part],
      directory=None,
      x_col="Img_Path",
      y_col="Label",
      weight_col=None,
      target_size=(224, 224),
      color_mode="rgb",
      classes=None,
      class_mode="categorical",
      batch_size=128,
      shuffle=False,
      seed=1,
      save_to_dir=None,
      save_prefix="",
      save_format="png",
      interpolation="nearest",
      validate_filenames=False
  )

  tta_steps = 10
  predictions = []

  for i in tqdm(range(tta_steps)):
      preds = model.predict(test_generator,verbose=1)
      predictions.append(preds)


  preds = np.mean(predictions, axis=0)

  voting_df=pd.DataFrame(data=np.c_[test_data[test_data.Body_Part==body_part]['Patient/Study'].values,preds],
                       columns=['Patient/Study','Class 0 prob','Class 1 prob'])
  
  voting_df[['Class 0 prob', 'Class 1 prob']] = voting_df[['Class 0 prob', 'Class 1 prob']].apply(pd.to_numeric)   
  voting_df=voting_df.groupby('Patient/Study').mean().reset_index()
  voting_df['Class_prediction']=np.argmax(voting_df.iloc[:,1:3].values,axis=1)
  voting_df=voting_df.merge(test_labels[test_labels.Body_Part==body_part],how='inner',on='Patient/Study')
  voting_df=voting_df.drop(columns=['Img_Path'])
  voting_df['Label'] = voting_df['Label'].apply(pd.to_numeric) 
  kappa_score=cohen_kappa_score(voting_df.Class_prediction,voting_df.Label)
  f1=f1_score(voting_df.Class_prediction,voting_df.Label)
  return [kappa_score,f1,voting_df]

### Train for each body part


### Train for each body part

#### ELBOW

In [ ]:
optimizer = Adam(1e-4)
sncallback_list = [EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,cooldown=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'+'best_model_densenet_XR_ELBOW'+'.h5', monitor='val_loss', save_best_only=True)]


dense169_model,dense169_hs = train_model(pooling='avg',
    body_type='XR_ELBOW',
    full_network_epochs=10,
    optimizer=optimizer,
    output_activation='softmax',
    callbacks_list=callback_list ,
    verbose=1)

clean_up(dense169_model)

# res_elbow=evaluate_model(body_part='XR_ELBOW',model=dense169_model)



  

#### FINGER

In [ ]:
optimizer = Adam(1e-4)
callback_list = [EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,cooldown=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'+'best_model_densenet_XR_FINGER'+'.h5', monitor='val_loss', save_best_only=True)]


dense169_model,dense169_hs = train_model(pooling='avg',
    body_type='XR_FINGER',
    full_network_epochs=10,
    optimizer=optimizer,
    output_activation='softmax',
    callbacks_list=callback_list ,
    verbose=1)

clean_up(dense169_model)



Found 4098 non-validated image filenames belonging to 2 classes.
Found 1008 non-validated image filenames belonging to 2 classes.
Epoch 1/10
128/128 [==============================] - 334s 3s/step - loss: 0.5637 - accuracy: 0.7113 - f1: 0.7134 - val_loss: 0.4345 - val_accuracy: 0.7006 - val_f1: 0.7006
Epoch 2/10
128/128 [==============================] - 255s 2s/step - loss: 0.4512 - accuracy: 0.7932 - f1: 0.7947 - val_loss: 0.4567 - val_accuracy: 0.7070 - val_f1: 0.7056
Epoch 3/10
128/128 [==============================] - 254s 2s/step - loss: 0.3861 - accuracy: 0.8306 - f1: 0.8306 - val_loss: 0.3524 - val_accuracy: 0.7777 - val_f1: 0.7792
Epoch 4/10
128/128 [==============================] - 251s 2s/step - loss: 0.3539 - accuracy: 0.8394 - f1: 0.8345 - val_loss: 0.2280 - val_accuracy: 0.8002 - val_f1: 0.8014
Epoch 5/10
128/128 [==============================] - 253s 2s/step - loss: 0.3026 - accuracy: 0.8741 - f1: 0.8713 - val_loss: 0.5720 - val_accuracy: 0.7193 - val_f1: 0.7177
Epoch

#### HAND

In [ ]:
optimizer = Adam(1e-4)
callback_list = [EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,cooldown=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'+'best_model_densenet_XR_HAND'+'.h5', monitor='val_loss', save_best_only=True)]


dense169_model,dense169_hs = train_model(pooling='avg',
    body_type='XR_HAND',
    full_network_epochs=10,
    optimizer=optimizer,
    output_activation='softmax',
    callbacks_list=callback_list ,
    verbose=1)




clean_up(dense169_model)


Found 4425 non-validated image filenames belonging to 2 classes.
Found 1118 non-validated image filenames belonging to 2 classes.
Epoch 1/10
138/138 [==============================] - 359s 3s/step - loss: 0.5493 - accuracy: 0.7530 - f1: 0.7537 - val_loss: 0.5048 - val_accuracy: 0.7454 - val_f1: 0.7454
Epoch 2/10
138/138 [==============================] - 277s 2s/step - loss: 0.4615 - accuracy: 0.7951 - f1: 0.7962 - val_loss: 0.4514 - val_accuracy: 0.7413 - val_f1: 0.7411
Epoch 3/10
138/138 [==============================] - 275s 2s/step - loss: 0.4152 - accuracy: 0.8193 - f1: 0.8202 - val_loss: 0.6811 - val_accuracy: 0.7716 - val_f1: 0.7716
Epoch 4/10
138/138 [==============================] - 275s 2s/step - loss: 0.3786 - accuracy: 0.8372 - f1: 0.8369 - val_loss: 0.6010 - val_accuracy: 0.8149 - val_f1: 0.8149

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 5/10
138/138 [==============================] - 274s 2s/step - loss: 0.2831 - accuracy: 0.8

#### HUMEROUS

In [ ]:
optimizer = Adam(1e-4)
callback_list = [EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,cooldown=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'+'best_model_densenet_XR_HUMERUS'+'.h5', monitor='val_loss', save_best_only=True)]


dense169_model,dense169_hs = train_model(pooling='avg',
    body_type='XR_HUMERUS',
    full_network_epochs=10,
    optimizer=optimizer,
    output_activation='softmax',
    callbacks_list=callback_list ,
    verbose=1)




clean_up(dense169_model)

Found 1056 non-validated image filenames belonging to 2 classes.
Found 216 non-validated image filenames belonging to 2 classes.
Epoch 1/10
33/33 [==============================] - 146s 4s/step - loss: 0.5645 - accuracy: 0.7178 - f1: 0.7178 - val_loss: 1.0798 - val_accuracy: 0.5625 - val_f1: 0.5625
Epoch 2/10
33/33 [==============================] - 67s 2s/step - loss: 0.3705 - accuracy: 0.8513 - f1: 0.8513 - val_loss: 0.5240 - val_accuracy: 0.7826 - val_f1: 0.7847
Epoch 3/10
33/33 [==============================] - 65s 2s/step - loss: 0.3307 - accuracy: 0.8665 - f1: 0.8665 - val_loss: 0.4300 - val_accuracy: 0.7826 - val_f1: 0.7795
Epoch 4/10
33/33 [==============================] - 65s 2s/step - loss: 0.2332 - accuracy: 0.9167 - f1: 0.9167 - val_loss: 0.4443 - val_accuracy: 0.8370 - val_f1: 0.8351
Epoch 5/10
33/33 [==============================] - 65s 2s/step - loss: 0.2025 - accuracy: 0.9205 - f1: 0.9205 - val_loss: 0.4988 - val_accuracy: 0.8152 - val_f1: 0.8142

Epoch 00005: Reduce

#### FOREARM

In [ ]:
optimizer = Adam(1e-4)
callback_list = [EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,cooldown=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'+'best_model_densenet_XR_FOREARM'+'.h5', monitor='val_loss', save_best_only=True)]


dense169_model,dense169_hs = train_model(pooling='avg',
    body_type='XR_FOREARM',
    full_network_epochs=10,
    optimizer=optimizer,
    output_activation='softmax',
    callbacks_list=callback_list ,
    verbose=1)




clean_up(dense169_model)


Found 1457 non-validated image filenames belonging to 2 classes.
Found 368 non-validated image filenames belonging to 2 classes.
Epoch 1/10
45/45 [==============================] - 175s 4s/step - loss: 0.5123 - accuracy: 0.7705 - f1: 0.7699 - val_loss: 0.7349 - val_accuracy: 0.7415 - val_f1: 0.7415
Epoch 2/10
45/45 [==============================] - 89s 2s/step - loss: 0.3598 - accuracy: 0.8618 - f1: 0.8620 - val_loss: 0.5425 - val_accuracy: 0.7202 - val_f1: 0.7131
Epoch 3/10
45/45 [==============================] - 89s 2s/step - loss: 0.2813 - accuracy: 0.8954 - f1: 0.8965 - val_loss: 0.7186 - val_accuracy: 0.7738 - val_f1: 0.7699
Epoch 4/10
45/45 [==============================] - 89s 2s/step - loss: 0.2430 - accuracy: 0.9074 - f1: 0.9083 - val_loss: 0.4719 - val_accuracy: 0.8244 - val_f1: 0.8153
Epoch 5/10
45/45 [==============================] - 89s 2s/step - loss: 0.1878 - accuracy: 0.9242 - f1: 0.9250 - val_loss: 1.1291 - val_accuracy: 0.8095 - val_f1: 0.7983
Epoch 6/10
45/45 [==

#### SHOULDER

In [ ]:
optimizer = Adam(1e-4)
callback_list = [EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,cooldown=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'+'best_model_densenet_XR_SHOULDER'+'.h5', monitor='val_loss', save_best_only=True)]


dense169_model,dense169_hs = train_model(pooling='avg',
    body_type='XR_SHOULDER',
    full_network_epochs=10,
    optimizer=optimizer,
    output_activation='softmax',
    callbacks_list=callback_list ,
    verbose=1)




clean_up(dense169_model)

Found 6716 non-validated image filenames belonging to 2 classes.
Found 1663 non-validated image filenames belonging to 2 classes.
Epoch 1/10
209/209 [==============================] - 513s 2s/step - loss: 0.5875 - accuracy: 0.7081 - f1: 0.7082 - val_loss: 0.6792 - val_accuracy: 0.6213 - val_f1: 0.6213
Epoch 2/10
209/209 [==============================] - 420s 2s/step - loss: 0.4992 - accuracy: 0.7721 - f1: 0.7722 - val_loss: 0.6438 - val_accuracy: 0.7069 - val_f1: 0.7069
Epoch 3/10
209/209 [==============================] - 418s 2s/step - loss: 0.4539 - accuracy: 0.7958 - f1: 0.7958 - val_loss: 0.4419 - val_accuracy: 0.7529 - val_f1: 0.7528
Epoch 4/10
209/209 [==============================] - 418s 2s/step - loss: 0.4116 - accuracy: 0.8167 - f1: 0.8168 - val_loss: 0.8639 - val_accuracy: 0.7039 - val_f1: 0.7039
Epoch 5/10
209/209 [==============================] - 418s 2s/step - loss: 0.3746 - accuracy: 0.8374 - f1: 0.8375 - val_loss: 0.5309 - val_accuracy: 0.7615 - val_f1: 0.7614

Epoc

#### WRIST

In [ ]:
optimizer = Adam(1e-4)
callback_list = [EarlyStopping(monitor='val_loss', patience=5, verbose=1,restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,cooldown=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'+'best_model_densenet_XR_WRIST'+'.h5', monitor='val_loss', save_best_only=True)]


dense169_model,dense169_hs = train_model(pooling='avg',
    body_type='XR_WRIST',
    full_network_epochs=10,
    optimizer=optimizer,
    output_activation='softmax',
    callbacks_list=callback_list ,
    verbose=1)




clean_up(dense169_model)

Found 7721 non-validated image filenames belonging to 2 classes.
Found 2031 non-validated image filenames belonging to 2 classes.
Epoch 1/10
241/241 [==============================] - 564s 2s/step - loss: 0.4858 - accuracy: 0.7757 - f1: 0.7760 - val_loss: 0.5279 - val_accuracy: 0.7634 - val_f1: 0.7634
Epoch 2/10
241/241 [==============================] - 484s 2s/step - loss: 0.3946 - accuracy: 0.8344 - f1: 0.8343 - val_loss: 0.3294 - val_accuracy: 0.8099 - val_f1: 0.8104
Epoch 3/10
241/241 [==============================] - 483s 2s/step - loss: 0.3500 - accuracy: 0.8547 - f1: 0.8545 - val_loss: 0.7731 - val_accuracy: 0.8249 - val_f1: 0.8253
Epoch 4/10
241/241 [==============================] - 483s 2s/step - loss: 0.3223 - accuracy: 0.8591 - f1: 0.8592 - val_loss: 0.5847 - val_accuracy: 0.8264 - val_f1: 0.8268

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 5/10
241/241 [==============================] - 483s 2s/step - loss: 0.2558 - accuracy: 0.8

### Load Model From Drive


In [ ]:
path_to_saved_models='/content/drive/My Drive/Deep Learning/Saved Models Project 2/'

In [ ]:
os.listdir(path_to_saved_models)

['best_model_freezed.h5',
 'best_model_densenet.h5',
 'XR_FOREARM_best_model_densenet.h5',
 'XR_HUMERUS_best_model_densenet.h5',
 'XR_FINGER_best_model_densenet.h5',
 'XR_ELBOW_best_model_densenet.h5',
 'XR_HAND_best_model_densenet.h5',
 'XR_SHOULDER_best_model_densenet.h5',
 'XR_WRIST_best_model_densenet.h5']

In [ ]:
load_model_list=os.listdir(path_to_saved_models)[2:]
bone_category=[]
for i in load_model_list:
  bone_category.append('_'.join(i.split('_')[0:2]))

In [ ]:
bone_category

['XR_FOREARM',
 'XR_HUMERUS',
 'XR_FINGER',
 'XR_ELBOW',
 'XR_HAND',
 'XR_SHOULDER',
 'XR_WRIST']

In [ ]:
loaded_models={}
for idx,model in zip(bone_category,load_model_list):
  print('Loading model {} ...'.format(idx))
  loaded_models[idx]=keras.models.load_model(path_to_saved_models+model,custom_objects={
        "f1":f1,
    })

Loading model XR_FOREARM ...
Loading model XR_HUMERUS ...
Loading model XR_FINGER ...
Loading model XR_ELBOW ...
Loading model XR_HAND ...
Loading model XR_SHOULDER ...
Loading model XR_WRIST ...


In [ ]:
eval_results={}
for k,v in loaded_models.items():
  eval_results[k]=evaluate_model(k,v)

  0%|          | 0/10 [00:00<?, ?it/s]

Found 301 non-validated image filenames belonging to 2 classes.
3/3 [==============================] - 3s 877ms/step


 10%|█         | 1/10 [00:10<01:37, 10.79s/it]

3/3 [==============================] - 3s 877ms/step


 20%|██        | 2/10 [00:17<01:17,  9.68s/it]

...

 80%|████████  | 8/10 [01:51<00:27, 13.76s/it]

6/6 [==============================] - 9s 1s/step


 90%|█████████ | 9/10 [02:05<00:13, 13.61s/it]

6/6 [==============================] - 9s 1s/step


100%|██████████| 10/10 [02:18<00:00, 13.85s/it]


In [ ]:
for k,v in eval_results.items():
  print('{}: Kappa Score {:.3f} f1 {:.3f}'.format(k,v[0],v[1]))

XR_FOREARM: Kappa Score 0.665 f1 0.796
XR_HUMERUS: Kappa Score 0.733 f1 0.859
XR_FINGER: Kappa Score 0.533 f1 0.706
XR_ELBOW: Kappa Score 0.682 f1 0.806
XR_HAND: Kappa Score 0.501 f1 0.648
XR_SHOULDER: Kappa Score 0.628 f1 0.806
XR_WRIST: Kappa Score 0.709 f1 0.810
